<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Batched-.match-operation" data-toc-modified-id="Batched-.match-operation-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Batched <code>.match</code> operation</a></span></li><li><span><a href="#More-examples" data-toc-modified-id="More-examples-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>More examples</a></span><ul class="toc-item"><li><span><a href="#Computing-matches-Euclidean-similarity" data-toc-modified-id="Computing-matches-Euclidean-similarity-0.2.1"><span class="toc-item-num">0.2.1&nbsp;&nbsp;</span>Computing matches Euclidean similarity</a></span></li><li><span><a href="#Computing-matches-Euclidean-distance" data-toc-modified-id="Computing-matches-Euclidean-distance-0.2.2"><span class="toc-item-num">0.2.2&nbsp;&nbsp;</span>Computing matches Euclidean distance</a></span></li><li><span><a href="#Computing-matches-Cosine-similarity" data-toc-modified-id="Computing-matches-Cosine-similarity-0.2.3"><span class="toc-item-num">0.2.3&nbsp;&nbsp;</span>Computing matches Cosine similarity</a></span></li></ul></li><li><span><a href="#test-distances-with-scipy" data-toc-modified-id="test-distances-with-scipy-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>test distances with scipy</a></span></li><li><span><a href="#Document-Memmap" data-toc-modified-id="Document-Memmap-0.4"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>Document Memmap</a></span><ul class="toc-item"><li><span><a href="#Test-distances-using-memmaparrays" data-toc-modified-id="Test-distances-using-memmaparrays-0.4.1"><span class="toc-item-num">0.4.1&nbsp;&nbsp;</span>Test distances using memmaparrays</a></span></li></ul></li></ul></li><li><span><a href="#Batching-distance-computation" data-toc-modified-id="Batching-distance-computation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Batching distance computation</a></span><ul class="toc-item"><li><span><a href="#Expressing-the-problem" data-toc-modified-id="Expressing-the-problem-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Expressing the problem</a></span></li><li><span><a href="#Building-the-function" data-toc-modified-id="Building-the-function-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Building the function</a></span></li><li><span><a href="#Adapting-batch-funtion-to-memmap-arrays" data-toc-modified-id="Adapting-batch-funtion-to-memmap-arrays-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Adapting batch funtion to memmap arrays</a></span></li><li><span><a href="#Benchmark-selection-of-best-values-from-a-distance-matrix" data-toc-modified-id="Benchmark-selection-of-best-values-from-a-distance-matrix-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Benchmark selection of best values from a distance matrix</a></span></li></ul></li></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2

In [78]:
from jina import Document, DocumentArray
import numpy as np

d1 = Document(embedding=np.array([0,0,0,0,1]))
d2 = Document(embedding=np.array([1,0,0,0,0]))
d3 = Document(embedding=np.array([1,1,1,1,0]))
d4 = Document(embedding=np.array([1,2,2,1,0]))

d1_m = Document(embedding=np.array([0,0.1,0,0,0]))
d2_m = Document(embedding=np.array([1,0.1,0,0,0]))
d3_m = Document(embedding=np.array([1,1.2,1,1,0]))
d4_m = Document(embedding=np.array([1,2.2,2,1,0]))
d5_m = Document(embedding=np.array([4,5.2,2,1,0]))

da_1 = DocumentArray([d1, d2, d3, d4])
da_2 = DocumentArray([d1_m, d2_m, d3_m, d4_m, d5_m])


da_1.match(da_2, metric='euclidean',  limit=3)
query = da_1[2]
print(f'query emb = {query.embedding}')
for m in query.matches:
    print('match emb =', m.embedding, 'score =', m.scores['euclidean'].value)

query emb = [1 1 1 1 0]
match emb = [1.  1.2 1.  1.  0. ] score = 0.20000000298023224
match emb = [1.  2.2 2.  1.  0. ] score = 1.5620499849319458
match emb = [1.  0.1 0.  0.  0. ] score = 1.6763054132461548


In [4]:
from jina import Document, DocumentArray
import scipy.sparse as sp

d1 = Document(embedding=sp.csr_matrix([0,0,0,0,1]))
d2 = Document(embedding=sp.csr_matrix([1,0,0,0,0]))
d3 = Document(embedding=sp.csr_matrix([1,1,1,1,0]))
d4 = Document(embedding=sp.csr_matrix([1,2,2,1,0]))

d1_m = Document(embedding=sp.csr_matrix([0,0.1,0,0,0]))
d2_m = Document(embedding=sp.csr_matrix([1,0.1,0,0,0]))
d3_m = Document(embedding=sp.csr_matrix([1,1.2,1,1,0]))
d4_m = Document(embedding=sp.csr_matrix([1,2.2,2,1,0]))
d5_m = Document(embedding=sp.csr_matrix([4,5.2,2,1,0]))

da_1 = DocumentArray([d1, d2, d3, d4])
da_2 = DocumentArray([d1_m, d2_m, d3_m, d4_m, d5_m])

da_1.match(da_2, metric='euclidean', limit=3)
query = da_1[2]
print(f'query emb = {query.embedding.todense()}')
for m in query.matches:
    print('match emb =', m.embedding.todense(), 'score =', m.scores['euclidean'].value)

TypeError: matrix is not square

## Batched `.match` operation

In [79]:
from jina import Document, DocumentArray
import numpy as np

d1 = Document(embedding=np.array([0,0,0,0,1]))
d2 = Document(embedding=np.array([1,0,0,0,0]))
d3 = Document(embedding=np.array([1,1,1,1,0]))
d4 = Document(embedding=np.array([1,2,2,1,0]))

da_1 = DocumentArray([d1, d2, d3, d4])

np.random.seed(1234)
x_mat = np.random.random((1000,5))
da_2 = DocumentArray([Document(embedding=x) for x in x_mat])

In [83]:
da_1.match(da_2, metric='euclidean',  limit=3)
query = da_1[2]
print(f'query emb = {query.embedding}')
for m in query.matches:
    print('match emb =', np.round(m.embedding,3), 'score =', round(m.scores['euclidean'].value,3))

query emb = [1 1 1 1 0]
match emb = [0.938 0.939 0.745 0.911 0.237] score = 0.37
match emb = [0.606 0.965 0.968 0.93  0.185] score = 0.443
match emb = [0.965 0.804 0.866 0.965 0.383] score = 0.454


In [88]:
da_1.match(da_2, metric='euclidean',  limit=3, batch_size=100)
query = da_1[2]
print(f'query emb = {query.embedding}')
for m in query.matches:
    print('match emb =', np.round(m.embedding,3), 'score =', round(m.scores['euclidean'].value,3))

query emb = [1 1 1 1 0]
match emb = [0.938 0.939 0.745 0.911 0.237] score = 0.37
match emb = [0.606 0.965 0.968 0.93  0.185] score = 0.443
match emb = [0.965 0.804 0.866 0.965 0.383] score = 0.454


In [94]:
!rm -rf dam2
dam_2 = DocumentArrayMemmap('./dam2')
dam_2.extend([Document(embedding=x) for x in x_mat])

da_1.match(dam_2, metric='euclidean',  limit=3, batch_size=100)
query = da_1[2]
print(f'query emb = {query.embedding}')
for m in query.matches:
    print('match emb =', np.round(m.embedding,3), 'score =', round(m.scores['euclidean'].value,3))

query emb = [1 1 1 1 0]
match emb = [0.938 0.939 0.745 0.911 0.237] score = 0.37
match emb = [0.606 0.965 0.968 0.93  0.185] score = 0.443
match emb = [0.965 0.804 0.866 0.965 0.383] score = 0.454


## More examples

In [9]:
from jina import Document, DocumentArray
import numpy as np
from jina.math.distance import cosine, sqeuclidean, cdist

In [10]:
n_features = 100

def get_document_arrays():
    d1 = Document(embedding=np.array([0,0,0,0,0]))
    d2 = Document(embedding=np.array([1,0,0,0,0]))
    d3 = Document(embedding=np.array([1,1,1,1,0]))
    d4 = Document(embedding=np.array([1,2,2,1,0]))

    d1_m = Document(embedding=np.array([0,0.1,0,0,0]))
    d2_m = Document(embedding=np.array([1,0.1,0,0,0]))
    d3_m = Document(embedding=np.array([1,1.2,1,1,0]))
    d4_m = Document(embedding=np.array([1,2.2,2,1,0]))
    d5_m = Document(embedding=np.array([4,5.2,2,1,0]))

    D  = DocumentArray([d1,d2,d3,d4])
    D2 = DocumentArray([d1_m, d2_m, d3_m, d4_m, d5_m])
    return D, D2

import scipy.sparse as sp

def get_document_arrays_sparse():
    d1 = Document(embedding=sp.csr_matrix([0,0,0,0,0]))
    d2 = Document(embedding=sp.csr_matrix([1,0,0,0,0]))
    d3 = Document(embedding=sp.csr_matrix([1,1,1,1,0]))
    d4 = Document(embedding=sp.csr_matrix([1,2,2,1,0]))

    d1_m = Document(embedding=sp.csr_matrix([0,0.1,0,0,0]))
    d2_m = Document(embedding=sp.csr_matrix([1,0.1,0,0,0]))
    d3_m = Document(embedding=sp.csr_matrix([1,1.2,1,1,0]))
    d4_m = Document(embedding=sp.csr_matrix([1,2.2,2,1,0]))
    d5_m = Document(embedding=sp.csr_matrix([4,5.2,2,1,0]))

    D  = DocumentArray([d1,d2,d3,d4])
    D2 = DocumentArray([d1_m, d2_m, d3_m, d4_m, d5_m])
    return D, D2

In [11]:
D, D2 = get_document_arrays()

In [12]:
E = np.stack(D.get_attributes('embedding'))
E.argsort(axis=1)

array([[0, 1, 2, 3, 4],
       [1, 2, 3, 4, 0],
       [4, 0, 1, 2, 3],
       [4, 0, 3, 1, 2]])

#### _compute_dist Euclidean distance

In [13]:
X = np.stack(D.get_attributes('embedding'))
Y = np.stack(D2.get_attributes('embedding'))

dists = cdist(X, Y, 'euclidean')
dists

array([[0.1       , 1.00498756, 2.10713075, 3.29241553, 6.93108938],
       [1.00498756, 0.1       , 1.8547237 , 3.13687743, 6.40624695],
       [1.95192213, 1.67630546, 0.2       , 1.56204994, 5.25737577],
       [3.1       , 2.93428015, 1.28062485, 0.2       , 4.38634244]])

### Computing matches Euclidean similarity

By default `D.match` uses a similarity metric.

This means matched results are ordered from highest to lowest

In [14]:
D, D2 = get_document_arrays()
D.match(D2, metric='euclidean', limit=3)

In [ ]:
print(f'query={D[2].embedding}')
for m in D[2].matches:
    print(m.embedding, 'score =', m.scores['euclidean'].value)

In [ ]:
print(f'query={D[1].embedding}')
for m in D[1].matches:
    print(m.embedding, 'score =', m.scores['euclidean'].value)

In [ ]:
from jina.enums import EmbeddingClsType

D, D2 = get_document_arrays_sparse()
D.match(D2, 
        metric='euclidean',
        limit=3,
        is_sparse=True)
        #embedding_cls_type = EmbeddingClsType.SCIPY_CSR)

In [ ]:
print(f'query={D[1].embedding.todense()}')
for m in D[1].matches:
    print(m.embedding.todense(), 'score =', m.scores['euclidean'].value)

In [ ]:
print(f'query={D[2].embedding.todense()}')
for m in D[2].matches:
    print(m.embedding.todense(), 'score =', m.scores['euclidean'].value)

### Computing matches Euclidean distance

If we want to use a distance we need to specify **`is_distance=True`**.

If we do so we should get results ordered from lowest to highest

In [ ]:
D, D2 = get_document_arrays()
D.match(D2, metric='euclidean', limit=3, is_distance=True)

In [ ]:
print(f'query={D[2].embedding}')
for m in D[2].matches:
    print(m.embedding, 'distance =', m.scores['euclidean'].value)

### Computing matches Cosine similarity

In [ ]:
D, D2 = get_document_arrays()
D.match(D2, metric='cosine', limit=3)

In [ ]:
print(f'query={D[2].embedding}')
for m in D[2].matches:
    print(m.embedding, 'dist=', m.scores['cosine'].value)

In [ ]:
scores = [m.scores['cosine'].value for m in D[2].matches]
scores ==sorted(scores)[::-1]

If we want cosine distance we need to specify `is_distance=True`

In [ ]:
D, D2 = get_document_arrays()
D.match(D2, metric='cosine', limit=3, is_distance=True)

In [ ]:
print(f'query={D[2].embedding}')
for m in D[2].matches:
    print(m.embedding, 'dist=', m.scores['cosine'].value)

In [ ]:
distances = [m.scores['cosine'].value for m in D[2].matches]
distances ==sorted(distances)

In [ ]:
[m.scores['cosine'].value for d in D for m in d.matches]

## test distances with scipy

In [ ]:

from scipy.spatial.distance import cdist

In [ ]:
#X = np.random.random((10,5))
#Y = np.random.random((15,5))

X = np.array([[1,1,1],[4,5,6],[0,1,2]])
Y = np.array([[1,1,2],[2,3,4]])
XYdistances = cdist(X,Y,metric='euclidean')
XY_new = np.sqrt(euclidean_distance_squared(X,Y))
np.testing.assert_almost_equal(XYdistances,XY_new)

In [ ]:
XYdistances = cdist(X,Y,metric='cosine')
XY_new = cosine_distance(X,Y)
np.testing.assert_almost_equal(XYdistances, XY_new)

In [ ]:
XY_new

In [ ]:
XYdistances

In [ ]:
X

In [ ]:
np.linalg.norm(X, axis=1)

In [ ]:
np.linalg.norm(Y, axis=1)

In [ ]:
np.outer(np.linalg.norm(X, axis=1), np.linalg.norm(Y, axis=1))  

In [ ]:
np.outer(np.array([1,2,3]), np.array([3,4]))

In [ ]:
2.44948974 * 1.73205081

## Document Memmap

In [ ]:
from jina.types.arrays.memmap import DocumentArrayMemmap

##### test_match_docarray_docarraymemmap

In [ ]:
metric = 'euclidean'
D1, D2 = get_document_arrays()
D1.match(D2, metric=metric, limit=3, is_distance=True)
values_docarray = [m.scores[metric].value for d in D1 for m in d.matches]

!rm -rf ./my-memmap
D1, D2 = get_document_arrays()
D2memmap = DocumentArrayMemmap('./my-memmap')
D2memmap.extend(D2)
D1.match(D2memmap, metric=metric, limit=3, is_distance=True)
values_docarraymemmap = [m.scores[metric].value for d in D1 for m in d.matches]

In [ ]:
values_docarraymemmap == values_docarraymemmap

##### test_docarraymemmap_match_docarray

In [ ]:
D1, D2 = get_document_arrays()
D1.match(D2, metric='euclidean', limit=3, is_distance=True)
values_docarray = [m.scores['euclidean'].value for d in D1 for m in d.matches]

!rm -rf ./my-memmap

from jina import Document
D1, D2 = get_document_arrays()
D1memmap = DocumentArrayMemmap('./my-memmap')
D1memmap.extend(D1)
D1memmap.match(D2, metric='euclidean', limit=3, is_distance=True)

print(f'query={dam[2].embedding}')
for m in D1memmap[2].matches:
    print(m.embedding, 'distance =', m.scores['euclidean'].value)
    
values_docarraymemmap = [m.scores['euclidean'].value for d in D1memmap for m in d.matches]

In [ ]:
values_docarray == values_docarraymemmap

##### test_docarraymemmap_match_docarraymemmap

In [ ]:
metric = 'euclidean'
is_distance = 'False'
D1, D2 = get_document_arrays()
D1.match(D2, metric=metric, limit=3, is_distance=True)
values_docarray = [m.scores[metric].value for d in D1 for m in d.matches]

!rm -rf ./my-memmap1
!rm -rf ./my-memmap2
D1, D2 = get_document_arrays()
D1memmap = DocumentArrayMemmap('./my-memmap1')
D1memmap.extend(D1)
D1memmap.reload()
D2memmap = DocumentArrayMemmap('./my-memmap2')
D2memmap.extend(D2)
D2memmap.reload()
D2memmap.reload()
D1memmap.match(D2memmap, metric=metric, limit=3, is_distance=is_distance)

#values_docarraymemmap = [m.scores[metric].value for d in D1memmap for m in d.matches]


In [ ]:
values_docarraymemmap = [m.scores[metric].value for d in D1memmap for m in d.matches]
values_docarraymemmap

In [ ]:
D1memmap[0].matches

In [ ]:
values_docarray

### Test distances using memmaparrays

In [15]:
euclidean_distance_squared(D1memmap, D2memmap)

NameError: name 'euclidean_distance_squared' is not defined

In [16]:
D1memmap.get_attributes('embedding')

NameError: name 'D1memmap' is not defined

# Batching distance computation

## Expressing the problem


Let `x_mat` and `y_mat` be two matrices with `n_x` and `n_y` rows respectively.

The ouput of our code is two matrices:

- `dists`: is a np.ndarray of shape `(n_x, k)` containing at row `i` the closest `k` distances from `x_mat[i,:]` and the `best` elements from `y_mat`. This matrix does not store which elements from `Y` are selected. This information is in the following matrix `inds`. 


- `inds`: is a np.ndarray of shape `(n_x, k)` containing at row `i` the indices of the `k` closest selected items from `y_mat`


We want to compute the pariwise distances between `x_mat` and `y_mat` assuming that rows in `x_mat` can be stored in memory but not all rows from `y_mat`.  Therefore we will load from `y_mat` in batches and compute the distances with each batch.


In [15]:
import jina.math 
from jina.math.distance import sqeuclidean
from jina.math.helper import top_k

## Building the function

In [16]:
n_features = 10
np.random.seed(1234)
X = np.random.random((10, n_features))
Y = np.random.random((1000,n_features))

print(X[0]) 
print(np.argpartition([4,0,2,6,9,1],kth=3))
print(sqeuclidean(X,Y).shape)

[0.19151945 0.62210877 0.43772774 0.78535858 0.77997581 0.27259261
 0.27646426 0.80187218 0.95813935 0.87593263]
[1 5 2 0 3 4]
(10, 1000)


In [17]:
dists, inds = top_k(sqeuclidean(X,Y), 10)
print(inds.shape, dists.shape)
print(dists[0])
print(inds[0])

(10, 10) (10, 10)
[0.22928135 0.24654283 0.26305431 0.34775188 0.35224021 0.37900384
 0.42007707 0.44362219 0.46516296 0.4734049 ]
[729 396 453 441 381  86 598 184 546 768]


In [18]:
def batch_generator(y_mat, n_batch):
    for i in range(0,len(y_mat), n_batch):
        yield y_mat[i:i+n_batch], i 

In [19]:
[(a[0].shape,a[1]) for a in batch_generator(Y, n_batch=100)]

[((100, 10), 0),
 ((100, 10), 100),
 ((100, 10), 200),
 ((100, 10), 300),
 ((100, 10), 400),
 ((100, 10), 500),
 ((100, 10), 600),
 ((100, 10), 700),
 ((100, 10), 800),
 ((100, 10), 900)]

In [35]:
def batch_top_k(x_mat, y_mat, batch_size, k):

    n_x = x_mat.shape[0]

    y_batch_generator = batch_generator(y_mat, batch_size)
    top_dists = np.inf * np.ones((n_x, k))
    top_inds  = np.zeros((n_x, k), dtype=int)
    
    for ybatch, ybatch_start_pos in y_batch_generator:
        distances = sqeuclidean(x_mat, ybatch)
        dists, inds = top_k(distances, k)
        inds = ybatch_start_pos + inds
        top_dists, top_inds = get_min_dists(dists, inds, top_dists, top_inds, k)
        
    # sort final the final `top_dists` and `top_inds` per row 
    perm = np.argsort(top_dists, axis=1)
    top_dists = np.take_along_axis(top_dists, perm, axis=1)
    top_inds = np.take_along_axis(top_inds, perm, axis=1)
    
    return top_dists, top_inds

In [36]:
def get_min_dists(dists, inds, top_dists, top_inds, k):
    all_dists = np.hstack((dists, top_dists))
    all_inds = np.hstack((inds, top_inds))
    best_inds = np.argpartition(all_dists, kth=k, axis=1)
    d_mat = np.take_along_axis(all_dists, best_inds , axis=1)
    i_mat = np.take_along_axis(all_inds, best_inds , axis=1)
    return d_mat[:,:k], i_mat[:,:k]

def get_min_dists2(dists, inds, top_dists, top_inds, k):
    all_dists = np.hstack((dists, top_dists))
    all_inds = np.hstack((inds, top_inds))
    best_inds = np.argpartition(all_dists, kth=k, axis=1)[:, 0:k]
    d_mat = np.array([all_dists[i,best_inds[i]] for i in range(len(all_dists))])
    i_mat = np.array([all_inds[i,best_inds[i]] for i in range(len(all_dists))])
    return d_mat, i_mat

In [37]:
n_features = 20
k = 5
batch_size = 100

np.random.seed(1234)
X = np.random.random((10, n_features))
Y = np.random.random((1000, n_features))

In [38]:
batch_top_k(X, Y, batch_size, k=k)

(array([[0.80362792, 1.04826136, 1.13512258, 1.21852459, 1.26330549],
        [1.04577613, 1.17615077, 1.34440863, 1.39320647, 1.42824177],
        [0.97770728, 1.26559054, 1.45667535, 1.48404514, 1.52595199],
        [1.11132182, 1.11401028, 1.27061648, 1.33421692, 1.34577427],
        [0.85772494, 0.97944147, 1.07031246, 1.13829755, 1.20326043],
        [1.39340144, 1.44160699, 1.45170949, 1.59847109, 1.73249643],
        [1.3281356 , 1.33286442, 1.39310716, 1.4341623 , 1.44763607],
        [1.03273538, 1.34571724, 1.34881996, 1.34959654, 1.44559504],
        [1.31508042, 1.51506797, 1.53971109, 1.55597349, 1.57407582],
        [0.94920786, 1.19361922, 1.23917019, 1.39217911, 1.40270516]]),
 array([[193, 951, 703, 294,  99],
        [311, 316, 291, 281, 304],
        [822,   2, 237, 197, 124],
        [657, 721, 462, 711, 560],
        [602, 312, 652, 215, 464],
        [544, 127, 623, 164, 448],
        [560, 203, 664, 511, 611],
        [987, 728, 339, 256, 479],
        [123, 890,

In [40]:
top_k(sqeuclidean(X,Y), k=k)

(array([[0.80362792, 1.04826136, 1.13512258, 1.21852459, 1.26330549],
        [1.04577613, 1.17615077, 1.34440863, 1.39320647, 1.42824177],
        [0.97770728, 1.26559054, 1.45667535, 1.48404514, 1.52595199],
        [1.11132182, 1.11401028, 1.27061648, 1.33421692, 1.34577427],
        [0.85772494, 0.97944147, 1.07031246, 1.13829755, 1.20326043],
        [1.39340144, 1.44160699, 1.45170949, 1.59847109, 1.73249643],
        [1.3281356 , 1.33286442, 1.39310716, 1.4341623 , 1.44763607],
        [1.03273538, 1.34571724, 1.34881996, 1.34959654, 1.44559504],
        [1.31508042, 1.51506797, 1.53971109, 1.55597349, 1.57407582],
        [0.94920786, 1.19361922, 1.23917019, 1.39217911, 1.40270516]]),
 array([[193, 951, 703, 294,  99],
        [311, 316, 291, 281, 304],
        [822,   2, 237, 197, 124],
        [657, 721, 462, 711, 560],
        [602, 312, 652, 215, 464],
        [544, 127, 623, 164, 448],
        [560, 203, 664, 511, 611],
        [987, 728, 339, 256, 479],
        [123, 890,

## Adapting batch funtion to memmap arrays 

In [41]:
def get_min_dists(dists, inds, top_dists, top_inds, k):
    all_dists = np.hstack((dists, top_dists))
    all_inds = np.hstack((inds, top_inds))
    best_inds = np.argpartition(all_dists, kth=k, axis=1)
    d_mat = np.take_along_axis(all_dists, best_inds , axis=1)
    i_mat = np.take_along_axis(all_inds, best_inds , axis=1)
    return d_mat[:,:k], i_mat[:,:k]

def batch_top_k(x_mat, y_mat, batch_size, k):

    n_x = x_mat.shape[0]

    y_batch_generator = batch_generator(y_mat, batch_size)
    top_dists = np.inf * np.ones((n_x, top_k_value))
    top_inds  = np.zeros((n_x, k), dtype=int)
    
    for ybatch, ybatch_start_pos in y_batch_generator:
        distances = sqeuclidean(x_mat, ybatch)
        dists, inds = top_k(distances, k)
        inds = ybatch_start_pos + inds
        top_dists, top_inds = get_min_dists(dists, inds, top_dists, top_inds, k)
        
    # sort final the final `top_dists` and `top_inds` per row 
    perm = np.argsort(top_dists, axis=1)
    top_dists = np.take_along_axis(top_dists, perm, axis=1)
    top_inds = np.take_along_axis(top_inds, perm, axis=1)
    
    return top_dists, top_inds

In [42]:
from jina.types.arrays.memmap import DocumentArrayMemmap
from jina import Document

n_features = 20
np.random.seed(1234)
X = np.random.random((10, n_features))
Y = np.random.random((1000, n_features))

y_dam = DocumentArrayMemmap('./my-memmap')
y_dam.extend([Document(embedding=y) for y in Y ])
y_dam[0:5]

TypeError: `key` must be int or str, but receiving slice(0, 5, None)

In [468]:
n_features = 20

np.random.seed(1234)
X = np.random.random((10, n_features))
Y = np.random.random((1000, n_features))

y_da = DocumentArray([Document(embedding=y) for y in Y ])

In [509]:
y_da[0:5]

<jina.types.arrays.document.DocumentArray length=5 at 140203653358256>

In [507]:
aux = np.memmap(y_dam._body)

In [508]:
len(aux)

1512000

## Benchmark selection of best values from a distance matrix

In [76]:
aux = np.vstack([y_dam[i].embedding for i in range(1000)])

In [77]:
all_dists = np.hstack((dists, top_dists))
all_inds = np.hstack((inds, top_inds))
best_inds = np.argpartition(all_dists, kth=top_k_value, axis=1)[:,:top_k_value]

NameError: name 'top_dists' is not defined

In [121]:
print(all_dists)
print(best_inds)
print(np.take_along_axis(all_dists, best_inds , axis=1))

[[0.3 0.1 0.6 0.2 0.5 0.9]
 [0.9 0.6 0.9 0.1 0.5 0.7]]
[[1 3 0]
 [3 4 1]]
[[0.1 0.2 0.3]
 [0.1 0.5 0.6]]


One way to do it is with a comprenhensive list

In [92]:
%%timeit
d_mat = np.array([all_dists[i,best_inds[i][:top_k_value]] for i in range(len(all_dists))])

4.67 µs ± 147 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Another way to do it is with `np.take_along_axis`

In [83]:
%%timeit
np.take_along_axis(all_dists, best_inds , axis=1)

6.48 µs ± 112 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Both methods return the same

In [82]:
np.array([all_dists[i,best_inds[i][:top_k_value]] for i in range(len(all_dists))])

array([[0.1, 0.2, 0.3],
       [0.1, 0.5, 0.6]])

In [80]:
np.take_along_axis(all_dists, best_inds , axis=1)

array([[0.1, 0.2, 0.3],
       [0.1, 0.5, 0.6]])

In [513]:
def f(a=1,b=2,c=None):
    if c:
        return a+b+c
    else:
        return a+b

In [518]:
f(1,2)

3

In [532]:
f(1,2,**{'d':24})

TypeError: f() got an unexpected keyword argument 'd'

In [533]:
def g(a=1,b=2):
    return a+b